# Extract PM2.5 Pixels and Convert Them to H3 Hexagons

### Motivation
Air quality monitoring and analysis are crucial for understanding the impact of particulate matter (PM2.5) on human health and the environment. However, working with large-scale raster datasets can be challenging due to their complex structure and limited compatibility with analytical tools. This document aims to address this by showing the data extraction process that converts PM2.5 raster data into H3 hexagons, enabling easier analysis and visualization.

### Dataset
The project will utilize the estimated annual and monthly ground-level fine particulate matter (PM2.5) dataset, spanning the years 1998 to 2021. This dataset combines Aerosol Optical Depth (AOD) retrievals from NASA MODIS, MISR, and SeaWIFS, generously provided by Washington University in St. Louis. As a demonstration, the project will focus on extracting and analyzing PM2.5 data for a single year and with less resolution (0.1° × 0.1°), while providing explanations on how to apply the same methodology to process the entire datasets (0.01° × 0.01° is also available).
https://sites.wustl.edu/acag/datasets/surface-pm2-5/

### Methodology
The proposed methodology involves two key steps:
1. The process will employ Python libraries such as xarray, netcdf4, and rasterio to extract the coordinates of raster pixels.
2. Once the pixel coordinates are obtained, vector pixels will be converted to H3 hexagons of a desired resolution.

In [1]:
import xarray as xa

from shapely.geometry import Polygon
from shapely import wkt

import pandas as pd
import geopandas as gpd

import h3

In [2]:
file = '../data/V5GL03.HybridPM25c_0p10.Global.202101-202112.nc'

In [3]:
ds = xa.open_dataset(file)
ds

<xarray.Dataset> Size: 18MB
Dimensions:  (lon: 3600, lat: 1230)
Coordinates:
  * lon      (lon) float64 29kB -179.9 -179.8 -179.8 ... 179.8 179.9 179.9
  * lat      (lat) float64 10kB -54.95 -54.85 -54.75 ... 67.75 67.85 67.95
Data variables:
    GWRPM25  (lat, lon) float32 18MB ...
Attributes:
    Title:            Hybrid PM_2_._5 [\mug/m^3]
    Contact:          Aaron van Donkelaar <aaron.vandonkelaar@wustl.edu>
    References:       van Donkelaar et al., Monthly Global Estimates of Fine ...
    Filename:         C:\Users\Aaron\Documents\MATLAB\PM25_wMelanie\GlobalMon...
    Format:           NetCDF-4
    Delta_Lat:        0.10000228881835938
    Delta_Lon:        0.0999908447265625
    SpatialCoverage:  Global
    Start_Date:       20210101
    End_Date:         20211231

Each datapoint has coordinates associated to it. Coordinates refer to grid centroids in latitude and longitude, and provided by the file metadata, we know that:

In [4]:
print('Pixel Longitude Side:', ds.Delta_Lon)
print('Pixel Latitude Side:', ds.Delta_Lat)

Pixel Longitude Side: 0.0999908447265625
Pixel Latitude Side: 0.10000228881835938


In [5]:
data = ds.to_dataframe().reset_index()
ds.close()

In [6]:
data = data.dropna()

In [8]:
def get_bounding_coordinates(lat, lon, delta_lat, delta_lon):
    half_side_lat = delta_lat/2
    half_side_lon = delta_lon/2

    point0 = (lon - half_side_lon, lat - half_side_lat)
    point1 = (lon + half_side_lon, lat - half_side_lat)
    point2 = (lon + half_side_lon, lat + half_side_lat)
    point3 = (lon - half_side_lon, lat + half_side_lat)

    return Polygon([point0, point1, point2, point3])

## Explanation of `get_bounding_coordinates` function

The `get_bounding_coordinates` function calculates the bounding coordinates (a pixel from the raster) of a given center point (latitude, longitude) based on the provided side lengths in the latitude and longitude directions.

### Function Parameters:
- `lat`: The latitude of the center point.
- `lon`: The longitude of the center point.
- `delta_lat`: The side length (in degrees) of the bounding box in the latitude direction.
- `delta_lon`: The side length (in degrees) of the bounding box in the longitude direction.

### Function Process:
1. `half_side_lat = delta_lat / 2`: Calculates half of the side length in the latitude direction. It's the distance from the center point to the top or bottom edge of the bounding box.

2. `half_side_lon = delta_lon / 2`: Calculates half of the side length in the longitude direction. It's the distance from the center point to the left or right edge of the bounding box.

3. Four points (`point0`, `point1`, `point2`, and `point3`) are defined based on the center point and the half side lengths:
   - `point0`: The bottom-left point of the bounding box.
   - `point1`: The bottom-right point of the bounding box.
   - `point2`: The top-right point of the bounding box.
   - `point3`: The top-left point of the bounding box.

4. `return Polygon([point0, point1, point2, point3])`: The function returns a `Polygon` object representing the bounding box, or the 'gridded pixel'. The `Polygon` is constructed using the four points calculated above, and it will be used to represent the region corresponding to the provided center point and pixel dimensions.


In [10]:
def filter_by_bounds(df, area_of_interest):
    # get minimum and maximum longitude and latitude values from gdf
    minx, miny, maxx, maxy = area_of_interest.bounds

    # filter df based on longitude and latitude
    df_filtered = df[
        (df["lon"] >= minx)
        & (df["lon"] <= maxx)
        & (df["lat"] >= miny)
        & (df["lat"] <= maxy)
    ]

    return df_filtered

We can use a bounding box to filter the data into a sample, easier to work in local. Check https://boundingbox.klokantech.com/, a tool to create polygons and obtain their WKT representation.

In [11]:
eu_bbox = 'POLYGON((-12.63 58.24, 30.08 58.24, 30.08 35.01, -12.63 35.01, -12.63 58.24))'
eu = wkt.loads(eu_bbox)

sample = filter_by_bounds(data, eu)

In [12]:
sample.loc[:, 'geometry'] = sample.apply(
    lambda row: get_bounding_coordinates(
        row['lat'], row['lon'], ds.Delta_Lat, ds.Delta_Lon
    ), axis = 1
)

/var/folders/2x/t13x4mn107zd1xc9j8ks6q300000gn/T/ipykernel_27476/869514934.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample.loc[:, 'geometry'] = sample.apply(


In [28]:
sample.head()

,lon,lat,GWRPM25,geometry
2084690,-10.549997,52.049999,7.1,POLYGON ((-10.599992752075195 51.9999980926513...
2085921,-10.449997,52.149998,7.0,POLYGON ((-10.499992370605469 52.0999965667724...
2087148,-10.349997,51.849998,7.1,POLYGON ((-10.399991989135742 51.7999973297119...
2087149,-10.349997,51.950001,7.1,POLYGON ((-10.399991989135742 51.8999996185302...
2087151,-10.349997,52.149998,7.6,POLYGON ((-10.399991989135742 52.0999965667724...


In [14]:
sample[['GWRPM25', 'geometry']]#.to_csv('pm25_pixels.csv', index=None)

,GWRPM25,geometry
2084690,7.1,POLYGON ((-10.599992752075195 51.9999980926513...
2085921,7.0,POLYGON ((-10.499992370605469 52.0999965667724...
2087148,7.1,POLYGON ((-10.399991989135742 51.7999973297119...
2087149,7.1,POLYGON ((-10.399991989135742 51.8999996185302...
2087151,7.6,POLYGON ((-10.399991989135742 52.0999965667724...
...,...,...
2584127,10.9,"POLYGON ((30.00000762939453 57.69999885559082,..."
2584128,10.7,POLYGON ((30.00000762939453 57.799997329711914...
2584129,10.8,"POLYGON ((30.00000762939453 57.89999961853027,..."
2584130,11.0,"POLYGON ((30.00000762939453 57.99999809265137,..."


Function that populates any geometry with the specified resolution hexagons. It return the h3 indexes which centroids fall inside the geometry passed.

In [22]:
h3.__version__

'4.0.0b2'

In [42]:
def H3PolygonPolyfill(unary, resolution):
	coords = [(lat, lon) for lon, lat in unary.exterior.coords]
	return h3.polygon_to_cells(h3.Polygon(coords), resolution)

The resolution parameter can be changed here:

In [44]:
RESOLUTION = 6

We iterate the H3 Polyfill function through all the rows of the data extracted from the NetCDF file.

In [45]:
hexes = []
for ix, row in sample.iterrows(), total = len(sample):
    hexes = list(H3PolygonPolyfill(row['geometry'], resolution = RESOLUTION))
    for h in hexes:
        newrow = [h, row['GWRPM25']]
        hexes.append(newrow)

hexes = pd.DataFrame(hexes, columns = ['hex','pm2.5'])

100%|██████████████████████████████████| 58171/58171 [00:03<00:00, 16516.32it/s]


You can use the next function to get the geometries for every hexagon in a dataframe:

In [ ]:
def load_hexes_geom(df):
	df['geometry'] = df['hex'].apply(lambda x: h3.cell_to_boundary(x, geo_json = True))
	df['geometry'] = df['geometry'].apply(Polygon)
	return gpd.GeoDataFrame(df)

In [49]:
# hexes.to_csv(f'h{RESOLUTION}_pm25.csv')